<a href="https://colab.research.google.com/github/kareemullah123456789/big_data_advanced/blob/main/section_4_broadcast_canada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [13]:
!ls /content/drive/MyDrive/cde_data/broadcast_logs


# Example: accessing a file
#file_path = '/content/drive/MyDrive/cde_data/broadcast_logs/your_file.csv'


 BroadcastLogs_2018_Q3_M8_sample.CSV  'data dictionary.doc'
 Call_Signs.csv			       ReferenceTables


In [18]:
!ls /content/drive/MyDrive/cde_data/broadcast_logs

 BroadcastLogs_2018_Q3_M8_sample.CSV  'data dictionary.doc'
 Call_Signs.csv			       ReferenceTables


In [28]:
"""Checkpoint code for the book Data Analysis with Python and PySpark, Chapter 4."""

import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

DIRECTORY = "/content/drive/MyDrive/cde_data/broadcast_logs"
logs = (
    spark.read.csv(
        os.path.join(DIRECTORY, "BroadcastLogs_2018_Q3_M8_sample.CSV"),
        sep="|",
        header=True,
        inferSchema=True,
        timestampFormat="yyyy-MM-dd",
    )
    .drop("BroadcastLogID", "SequenceNO")
    .withColumn(
        "duration_seconds",
        (
            F.col("Duration").substr(1, 2).cast("int") * 60 * 60
            + F.col("Duration").substr(4, 2).cast("int") * 60
            + F.col("Duration").substr(7, 2).cast("int")
        ),
    )
)

logs.show()

+------------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+--------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+----------------+
|LogServiceID|   LogDate|AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|EthnicProgramID|ProductionSourceID|ProgramClassID|FilmClassificationID|ExhibitionID|        Duration|         EndTime|LogEntryDate|ProductionNO|        ProgramTitle|       StartTime|Subtitle|NetworkAffiliationID|SpecialAttentionID|BroadcastOriginPointID|CompositionID|Producer1|Producer2|Language1|Language2|duration_seconds|
+------------+----------+-------------------+----------------------+----------+---------

In [20]:
!ls /content/drive/MyDrive/cde_data/broadcast_logs

 BroadcastLogs_2018_Q3_M8_sample.CSV  'data dictionary.doc'
 Call_Signs.csv			       ReferenceTables


In [30]:
#  commercials.py #############################################################
#
# This program computes the commercial ratio for each channel present in the
# dataset.
#
###############################################################################

import os

import pyspark.sql.functions as F
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName(
    "Getting the Canadian TV channels with the highest/lowest proportion of commercials."
).getOrCreate()

spark.sparkContext.setLogLevel("WARN")

###############################################################################
# Reading all the relevant data sources
###############################################################################

DIRECTORY = "/content/drive/MyDrive/cde_data/broadcast_logs"

logs = spark.read.csv(
    os.path.join(DIRECTORY, "BroadcastLogs_2018_Q3_M8_sample.CSV"),
    sep="|",
    header=True,
    inferSchema=True,
)
logs.printSchema()


root
 |-- BroadcastLogID: integer (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: date (nullable = true)
 |-- SequenceNO: integer (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: timestamp (nullable = true)
 |-- EndTime: timestamp (nullable = true)
 |-- LogEntryDate: date (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: timestamp (nullable = true)
 |-- Subtitle: string (

In [31]:
log_identifier = spark.read.csv(
    "/content/drive/MyDrive/cde_data/broadcast_logs/ReferenceTables/LogIdentifier.csv",
    sep="|",
    header=True,
    inferSchema=True,
)

log_identifier.show()

+---------------+------------+---------+
|LogIdentifierID|LogServiceID|PrimaryFG|
+---------------+------------+---------+
|           13ST|        3157|        1|
|         2000SM|        3466|        1|
|           70SM|        3883|        1|
|           80SM|        3590|        1|
|           90SM|        3470|        1|
|         9DAPTN|        3158|        1|
|         9DCFCF|        3159|        1|
|         9DCFRN|        3160|        1|
|         9DCHRO|        3161|        1|
|         9DCIVI|        3162|        1|
|         9DCJCH|        3163|        1|
|         9DCJOH|        3164|        1|
|         9DCKCK|        3165|        1|
|         9DCKCW|        3166|        1|
|         9DCKVR|        3167|        1|
|          9DCKY|        3168|        1|
|         9DLEAF|        3169|        1|
|          9DMC2|        3170|        1|
|         9DMPIX|        3171|        1|
|         9DMUCH|        3172|        1|
+---------------+------------+---------+
only showing top

In [33]:
cd_category = spark.read.csv(
    "/content/drive/MyDrive/cde_data/broadcast_logs/ReferenceTables/CD_Category.csv",
    sep="|",
    header=True,
    inferSchema=True,
).select(
    "CategoryID",
    "CategoryCD",
    F.col("EnglishDescription").alias("Category_Description"),
)

cd_category.printSchema()

root
 |-- CategoryID: integer (nullable = true)
 |-- CategoryCD: string (nullable = true)
 |-- Category_Description: string (nullable = true)



In [34]:
cd_category.show()

+----------+----------+--------------------+
|CategoryID|CategoryCD|Category_Description|
+----------+----------+--------------------+
|         1|       010|                NEWS|
|         2|       02 |CANREC  ANALYSIS ...|
|         3|       02A|ANALYSIS AND INTE...|
|         4|       02B|LONG-FORM DOCUMEN...|
|         5|       030|REPORTING & ACTUA...|
|         6|       040|            RELIGION|
|         7|       05A|    EDUCATION FORMAL|
|         8|       05B|  EDUCATION INFORMAL|
|         9|       06A|SPORTS - PROFESSI...|
|        10|       06B|SPORTS - AMATEUR ...|
|        11|       07A|ONGOING DRAMATIC ...|
|        12|       07B|ONGOING COMEDY SE...|
|        13|       07C|SPECIALS,MINI-SER...|
|        14|       07D|DRAMA- FEATURE FILMS|
|        15|       07E|     DRAMA- ANIMATED|
|        16|       07F|PROGRAMS OF COMED...|
|        17|       07G|               OTHER|
|        18|       08A|       MUSIC - DANCE|
|        19|       08B|MUSIC - DANCE - M...|
|        2

In [35]:




cd_program_class = spark.read.csv(
    "/content/drive/MyDrive/cde_data/broadcast_logs/ReferenceTables/CD_ProgramClass.csv",
    sep="|",
    header=True,
    inferSchema=True,
).select(
    "ProgramClassID",
    "ProgramClassCD",
    F.col("EnglishDescription").alias("ProgramClass_Description"),
)


cd_program_class.show()

+--------------+--------------+------------------------+
|ProgramClassID|ProgramClassCD|ProgramClass_Description|
+--------------+--------------+------------------------+
|             1|          AUT |           AUTOPROMOTION|
|             2|          BAL |     BALANCE PROGRAMMING|
|             3|          COM |      COMMERCIAL MESSAGE|
|             4|          COR |             CORNERSTONE|
|             5|          DOC |             DOCUMENTARY|
|             6|          GIA |               GIVEAWAYS|
|             7|          ID  |    NETWORK IDENTIFIC...|
|             8|          INT |            INTERSTITIAL|
|             9|          LOC |       LOCAL ADVERTISING|
|            10|          MAG |        MAGAZINE PROGRAM|
|            11|          MER |           MERCHANDISING|
|            12|          MVC |        MUSIC VIDEO CLIP|
|            13|          NRN |    No recognized nat...|
|            14|          OFF |    SCHEDULED OFF AIR...|
|            15|          OFT |

In [36]:
###############################################################################
# Data processing
###############################################################################

logs = logs.drop("BroadcastLogID", "SequenceNO")

logs = logs.withColumn(
    "duration_seconds",
    (
        F.col("Duration").substr(1, 2).cast("int") * 60 * 60
        + F.col("Duration").substr(4, 2).cast("int") * 60
        + F.col("Duration").substr(7, 2).cast("int")
    ),
)


logs.show()

+------------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+-------------------+-------------------+------------+------------+--------------------+-------------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+----------------+
|LogServiceID|   LogDate|AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|EthnicProgramID|ProductionSourceID|ProgramClassID|FilmClassificationID|ExhibitionID|           Duration|            EndTime|LogEntryDate|ProductionNO|        ProgramTitle|          StartTime|Subtitle|NetworkAffiliationID|SpecialAttentionID|BroadcastOriginPointID|CompositionID|Producer1|Producer2|Language1|Language2|duration_seconds|
+------------+----------+-------------------+----------------------+--

In [37]:
log_identifier = log_identifier.where(F.col("PrimaryFG") == 1)

In [38]:
log_identifier.show()

+---------------+------------+---------+
|LogIdentifierID|LogServiceID|PrimaryFG|
+---------------+------------+---------+
|           13ST|        3157|        1|
|         2000SM|        3466|        1|
|           70SM|        3883|        1|
|           80SM|        3590|        1|
|           90SM|        3470|        1|
|         9DAPTN|        3158|        1|
|         9DCFCF|        3159|        1|
|         9DCFRN|        3160|        1|
|         9DCHRO|        3161|        1|
|         9DCIVI|        3162|        1|
|         9DCJCH|        3163|        1|
|         9DCJOH|        3164|        1|
|         9DCKCK|        3165|        1|
|         9DCKCW|        3166|        1|
|         9DCKVR|        3167|        1|
|          9DCKY|        3168|        1|
|         9DLEAF|        3169|        1|
|          9DMC2|        3170|        1|
|         9DMPIX|        3171|        1|
|         9DMUCH|        3172|        1|
+---------------+------------+---------+
only showing top

In [39]:
logs.show()

+------------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+-------------------+-------------------+------------+------------+--------------------+-------------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+----------------+
|LogServiceID|   LogDate|AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|EthnicProgramID|ProductionSourceID|ProgramClassID|FilmClassificationID|ExhibitionID|           Duration|            EndTime|LogEntryDate|ProductionNO|        ProgramTitle|          StartTime|Subtitle|NetworkAffiliationID|SpecialAttentionID|BroadcastOriginPointID|CompositionID|Producer1|Producer2|Language1|Language2|duration_seconds|
+------------+----------+-------------------+----------------------+--

In [40]:


logs_and_channels = logs.join(log_identifier, "LogServiceID")

In [41]:
logs_and_channels.printSchema()

root
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: date (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: timestamp (nullable = true)
 |-- EndTime: timestamp (nullable = true)
 |-- LogEntryDate: date (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: timestamp (nullable = true)
 |-- Subtitle: string (nullable = true)
 |-- NetworkAffiliationID: integer (nullable = true)
 |-- SpecialAttentio

In [42]:
full_log = logs_and_channels.join(cd_category, "CategoryID", how="left").join(
    cd_program_class, "ProgramClassID", how="left"
)

full_log.show()

+--------------+----------+------------+----------+-------------------+----------------------+---------------+-----------------+----------------+---------------+------------------+--------------------+------------+-------------------+-------------------+------------+------------+--------------------+-------------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+----------------+---------------+---------+----------+--------------------+--------------+------------------------+
|ProgramClassID|CategoryID|LogServiceID|   LogDate|AudienceTargetAgeID|AudienceTargetEthnicID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|EthnicProgramID|ProductionSourceID|FilmClassificationID|ExhibitionID|           Duration|            EndTime|LogEntryDate|ProductionNO|        ProgramTitle|          StartTime|Subtitle|NetworkAffiliationID|SpecialAttentionID|BroadcastOriginPointID|CompositionID|Producer1|Producer2|Language1|

In [43]:




full_log.groupby("LogIdentifierID").agg(
    F.sum(
        F.when(
            F.trim(F.col("ProgramClassCD")).isin(
                ["COM", "PRC", "PGI", "PRO", "LOC", "SPO", "MER", "SOL"]
            ),
            F.col("duration_seconds"),
        ).otherwise(0)
    ).alias("duration_commercial"),
    F.sum("duration_seconds").alias("duration_total"),
).withColumn(
    "commercial_ratio", F.col("duration_commercial") / F.col("duration_total")
).orderBy(
    "commercial_ratio", ascending=False
).show(
    1000, False
)

+---------------+-------------------+--------------+----------------+
|LogIdentifierID|duration_commercial|duration_total|commercial_ratio|
+---------------+-------------------+--------------+----------------+
|CJCO           |0                  |NULL          |NULL            |
|BRAVO          |0                  |NULL          |NULL            |
|CFTF           |0                  |NULL          |NULL            |
|CKCS           |0                  |NULL          |NULL            |
|CJNT           |0                  |NULL          |NULL            |
|CKES           |0                  |NULL          |NULL            |
|CHBX           |0                  |NULL          |NULL            |
|BBCKID         |0                  |NULL          |NULL            |
|BOOK           |0                  |NULL          |NULL            |
|CHAN           |0                  |NULL          |NULL            |
|CEVASI         |0                  |NULL          |NULL            |
|CMT            |0  